In [82]:
import anthropic

In [83]:
from dotenv import load_dotenv

In [84]:
load_dotenv()

True

In [85]:
client_anthropic = anthropic.Anthropic(api_key = os.getenv('CLAUDE_API_KEY'))

In [86]:
import os

In [87]:
import json

In [88]:
from supabase import create_client, Client

In [89]:
supabase_api_key = os.environ.get("SUPBASE_KEY")
supabase_url = os.environ.get("SUPABASE_URL")

In [90]:
supabase: Client = create_client(supabase_url, supabase_api_key)

In [91]:
researched_data = [{'event_id': '299_2026-02-08',
  'news_date': '2026-02-08',
  'output': '- Verified: The earbuds rumor centers on an internal code name for OpenAI’s first consumer hardware device being Sweetpea. Multiple credible outlets report this codename as part of the leak cycle about the device. ([techcrunch.com](https://techcrunch.com/2026/01/21/openai-aims-to-ship-its-first-device-in-2026-and-it-could-be-earbuds/))\n\n- Verified: Several credible outlets note that the project is not officially confirmed and remains a leak/rumor at this time. TechCrunch explicitly describes the details as coming from leaks and “tipsters,” not an official OpenAI confirmation; Axios likewise characterizes the information as rumors shared at Davos. ([techcrunch.com](https://techcrunch.com/2026/01/21/openai-aims-to-ship-its-first-device-in-2026-and-it-could-be-earbuds/))\n\n- Verified: The same reporting that mentions Sweetpea also references a potential second consumer device codenamed Gumdrop in the rumor mill. ([axios.com](https://www.axios.com/2026/01/23/openai-device-earbuds-sweetpea-altman))\n\n- Added confirmed hardware context: OpenAI disclosed at Davos that it is building a new AI device, though details remain scarce. Timeline given by OpenAI officials in these discussions points to a likely release in the second half of 2026. ([axios.com](https://www.axios.com/2026/01/23/openai-device-earbuds-sweetpea-altman))\n\n- Confirmed: OpenAI’s prior hardware efforts are linked to its 2025 acquisition of Jony Ive’s design firm io (the deal was reported to be around $6.5 billion). Ive is associated with leading the hardware design work for the product family. ([techcrunch.com](https://techcrunch.com/2026/01/21/openai-aims-to-ship-its-first-device-in-2026-and-it-could-be-earbuds/))\n\n- Confirmed: OpenAI’s hardware plans discussed publicly have included a screen-free, pocketable device concept as part of the early product narrative. ([techcrunch.com](https://techcrunch.com/2026/01/21/openai-aims-to-ship-its-first-device-in-2026-and-it-could-be-earbuds/))\n\n- Confirmed: Reported design descriptions for the Sweetpea concept include a main unit described as metal and egg-shaped, with two pill-shaped modules that would sit behind the ear. These details appear in multiple outlets drawing on the same leaks. ([indianexpress.com](https://indianexpress.com/article/technology/tech-news-technology/openai-x-jony-ive-wireless-earbuds-unique-design-launch-2026-10471143/))\n\n- Confirmed (design details corroborated by multiple outlets): The rumored hardware would reportedly use a 2-nanometer-class processor (with Exynos as a favored reference) for on-device AI tasks, aligned with the “phone-like” compute ambitions discussed in the leaks. ([indianexpress.com](https://indianexpress.com/article/technology/tech-news-technology/openai-x-jony-ive-wireless-earbuds-unique-design-launch-2026-10471143/))\n\n- Confirmed (manufacturing partner context): Reports mention Foxconn as a manufacturing partner for the OpenAI devices, with some mention of Luxshare as a possibility; this is described in the leakage coverage and echoed by outlets citing Asian reporting. ([techcrunch.com](https://techcrunch.com/2026/01/21/openai-aims-to-ship-its-first-device-in-2026-and-it-could-be-earbuds/))\n\n- Confirmed: First-year shipment targets cited in the leaks are in the 40–50 million units range, with September 2026 as a possible launch window in at least some outlets. ([techcrunch.com](https://techcrunch.com/2026/01/21/openai-aims-to-ship-its-first-device-in-2026-and-it-could-be-earbuds/))\n\n- Confirmed (scope/roadmap detail in leaks): Some leakage coverage claims OpenAI is planning a broader hardware roadmap (up to five devices by 2028), with Sweetpea at the front of that lineup. This detail appears in multiple leak-focused outlets. ([androidauthority.com](https://www.androidauthority.com/openai-earbuds-3631925/))\n\n- Not corroborated by credible outlets (branding name caveat): The claim that the consumer-facing name will be “Dime” lacks confirmation from established major outlets. Some blogs and tech sites have floated “Dime” as branding based on patent filings or rumor chatter, but there is no widely credited report in outlets like TechCrunch, Axios, or The Indian Express confirming “Dime.” Examples of circulating but less-established sources include Gizmochina and other aggregators. ([gizmochina.com](https://www.gizmochina.com/2026/02/07/openais-first-hardware-device-ai-earbuds-strategy-shift-and-launch-timeline/?utm_source=openai))\n\n- Note on the current status of confirmation: The available reporting from January–February 2026 consistently describes the information as leaks/rumors rather than official confirmation; OpenAI executives acknowledged work on a hardware device but kept specifics (design, branding, features) under wraps. ([techcrunch.com](https://techcrunch.com/2026/01/21/openai-aims-to-ship-its-first-device-in-2026-and-it-could-be-earbuds/))\n\nSources referenced above (examples of main reporting):\n- TechCrunch: OpenAI aims to ship its first device in 2026, and it could be earbuds (codename Sweetpea; design, chip, and unit targets discussed). ([techcrunch.com](https://techcrunch.com/2026/01/21/openai-aims-to-ship-its-first-device-in-2026-and-it-could-be-earbuds/))\n- Indian Express: OpenAI’s first product could be ChatGPT-powered earbuds; codename Sweetpea; design details and 2nm Samsung Exynos chip mentioned. ([indianexpress.com](https://indianexpress.com/article/technology/tech-news-technology/openai-x-jony-ive-wireless-earbuds-unique-design-launch-2026-10471143/))\n- Android Authority: OpenAI earbuds leak with Sweetpea codename; design description; BOM/production cost hints; September launch target; five-device roadmap mentioned. ([androidauthority.com](https://www.androidauthority.com/openai-earbuds-3631925/))\n- Axios: OpenAI’s secret device coming; Davos timeline; mentions Sweetpea and Gumdrop leaks; timeline for second half of 2026. ([axios.com](https://www.axios.com/2026/01/23/openai-device-earbuds-sweetpea-altman))\n- The Verge: OpenAI hardware prototype discussions tied to Jony Ive and the hardware program (context around the hardware push). ([theverge.com](https://www.theverge.com/news/827607/openai-hardware-prototype-chatgpt-jony-ive-sam-altman?utm_source=openai))\n- (Caveat on Dime branding) Gizmochina / other aggregators noting a claimed consumer name Dime; these are not widely corroborated by major outlets. ([gizmochina.com](https://www.gizmochina.com/2026/02/07/openais-first-hardware-device-ai-earbuds-strategy-shift-and-launch-timeline/?utm_source=openai))\n\nNotes:\n- All statements above about Sweetpea, the rumored features, and the general timeline are based on published reporting as of February 8, 2026; several items (notably the consumer-name “Dime”) are not yet verified by major, high-profile outlets and are reported only by a subset of non-primary sources. ([techcrunch.com](https://techcrunch.com/2026/01/21/openai-aims-to-ship-its-first-device-in-2026-and-it-could-be-earbuds/))\n\nIf you’d like, I can add direct excerpts or align these points to a formal, source-by-source bibliography.'}]

In [93]:
result = supabase.table('research_assistant')\
    .select('id, event_id, news_date, output')\
.eq('news_date', '2026-02-20')\
    .order('created_at', desc=False)\
    .execute()

In [94]:
result.data

[{'id': 211,
  'event_id': '1274_2026-02-20',
  'news_date': '2026-02-20',
  'output': '- Core development: Nvidia is close to finalizing roughly a $30 billion equity investment in OpenAI, replacing the previously announced up-to-$100 billion multi-year data-center compute pact. ([thestar.com.my](https://www.thestar.com.my/tech/tech-news/2026/02/20/nvidia-openai-near-30-billion-investment-in-place-of-unfinished-100-billion-deal-ft-reports))\n\n- Financing scope: OpenAI is seeking more than $100 billion in the current funding round, with the round valuing OpenAI at about $830 billion. ([thestar.com.my](https://www.thestar.com.my/tech/tech-news/2026/02/20/nvidia-openai-near-30-billion-investment-in-place-of-unfinished-100-billion-deal-ft-reports))\n\n- Likely investors: SoftBank and Amazon are also expected to participate in the round. ([thestar.com.my](https://www.thestar.com.my/tech/tech-news/2026/02/20/nvidia-openai-near-30-billion-investment-in-place-of-unfinished-100-billion-deal-ft

In [95]:
final_summary_7 = []

In [97]:
def claude_article_builder(researched_data):
    final_news_article = []
    for i in researched_data:
        response = client_anthropic.messages.create(
            model = "claude-sonnet-4-5",
            max_tokens = 800,
            messages = [{
                "role": "user",
                "content":  f"""
                You are a tech journalist at Krux — a news platform that helps GenZ stay informed about AI & tech in under 60 seconds.

                CRITICAL CONSTRAINT: The article body MUST be 85-100 words (excluding headline). Count carefully before finalizing.

                TASK: Write a news article using ONLY the information provided by our research assistant.
                
                Follow this structure:
                
                === STEP 1: HEADLINE (Under 10 words) ===
                Objective: Hook the reader instantly
                - Punchy, attention-grabbing and clickbait
                - Focus on the most surprising/important angle
                
                **ARTICLE BODY** (85-100 words TOTAL)

                Opening (20-30 words):
                - If rumour: clarify in first sentence, then proceed assuming the things stated by research assistant are facts
                - Lead with the core news hook
                
                Middle (40-50 words):
                - Sharp, conversational, slightly witty tone
                - Present key facts from research assistant
                - Use most/all provided information
                - NO links or source references in body
                - Explain it in a simple way such that a non-tech person who has no background about this also understands its impact
                
                Closing (20-25 words):
                - Forward-looking strategic perspective
                - What could this mean for industry/users/tech landscape?
                - Frame as analysis, NOT opinion
                - Grounded in facts presented
                
                === VERIFICATION CHECKLIST ===
                Before outputting, verify:
                ☑ Article body is 85-100 words (count manually)
                ☑ Headline is under 10 words
                ☑ No URLs in article body
                ☑ All key facts from research included
                ☑ There should be no em-dashes in the article
                
                HARD CONSTRAINTS:
                - Total article: 100 words max (headline excluded). If the Article is NOT 100 words, re-generate it to meet the 100 word
                requirement.
                - Sources with URLs listed at end under "Sources:"

                
                OUTPUT FORMAT NEEDS TO BE IN A STRICT JSON FORMAT:
                {{"headline": "...", "output": "...", "sources": [{{"name": "...", "url": "..."}}]}}

                Here is the information we have gathered from the research assistant:
                
                {i['output']}
            """
            }]
        )

        final_output = response.content[0].text

        cleaned = final_output.strip()

        if cleaned.startswith("```"):
            cleaned = cleaned.split("\n", 1)[1].rsplit("```", 1)[0].strip()

        article_json = json.loads(cleaned)

        article_json['model_provider'] = 'claude'
        article_json['news_date'] = i['news_date']
        article_json['event_id'] = i['event_id']

        print(article_json)

        final_summary_7.append(article_json)

        save_article(article_json)

        print("✔️ Saved!")
        
        

    #     final_news_article.append(json_format)

    # return final_news_article

In [99]:
a = [{'id': 211,
  'event_id': '1274_2026-02-20',
  'news_date': '2026-02-20',
  'output': '- Core development: Nvidia is close to finalizing roughly a $30 billion equity investment in OpenAI, replacing the previously announced up-to-$100 billion multi-year data-center compute pact. ([thestar.com.my](https://www.thestar.com.my/tech/tech-news/2026/02/20/nvidia-openai-near-30-billion-investment-in-place-of-unfinished-100-billion-deal-ft-reports))\n\n- Financing scope: OpenAI is seeking more than $100 billion in the current funding round, with the round valuing OpenAI at about $830 billion. ([thestar.com.my](https://www.thestar.com.my/tech/tech-news/2026/02/20/nvidia-openai-near-30-billion-investment-in-place-of-unfinished-100-billion-deal-ft-reports))\n\n- Likely investors: SoftBank and Amazon are also expected to participate in the round. ([thestar.com.my](https://www.thestar.com.my/tech/tech-news/2026/02/20/nvidia-openai-near-30-billion-investment-in-place-of-unfinished-100-billion-deal-ft-reports))\n\n- Investment structure: The deal would be a direct equity stake in OpenAI, not a commitment for OpenAI to buy Nvidia hardware under a compute arrangement. ([thestar.com.my](https://www.thestar.com.my/tech/tech-news/2026/02/20/nvidia-openai-near-30-billion-investment-in-place-of-unfinished-100-billion-deal-ft-reports))\n\n- Use of proceeds: OpenAI plans to reinvest much of the new capital into Nvidia hardware and to purchase Nvidia chips. ([thestar.com.my](https://www.thestar.com.my/tech/tech-news/2026/02/20/nvidia-openai-near-30-billion-investment-in-place-of-unfinished-100-billion-deal-ft-reports))\n\n- Replacement of prior pact: This arrangement would replace the September announcement of a up-to-$100 billion multi-year commitment to support OpenAI’s use of Nvidia chips in data centers. ([thestar.com.my](https://www.thestar.com.my/tech/tech-news/2026/02/20/nvidia-openai-near-30-billion-investment-in-place-of-unfinished-100-billion-deal-ft-reports))\n\n- Company response: Nvidia declined to comment on the Reuters report. ([thestar.com.my](https://www.thestar.com.my/tech/tech-news/2026/02/20/nvidia-openai-near-30-billion-investment-in-place-of-unfinished-100-billion-deal-ft-reports))\n\n- Timing and status: The Reuters report notes the deal is not final and could close soon, with coverage dated February 19–20, 2026. ([thestar.com.my](https://www.thestar.com.my/tech/tech-news/2026/02/20/nvidia-openai-near-30-billion-investment-in-place-of-unfinished-100-billion-deal-ft-reports))'},
 {'id': 212,
  'event_id': '1275_2026-02-20',
  'news_date': '2026-02-20',
  'output': '- Core development: OpenAI and Tata Group announce a multi-dimensional strategic partnership to speed AI-driven transformation across Tata Group, other Indian and global enterprises, and to build AI infrastructure in India (Enterprise ChatGPT for Tata, agentic AI, and HyperVault data-center initiatives). ([tata.com](https://www.tata.com/newsroom/business/tata-openai-partnership))\n\n- OpenAI for India launch: OpenAI launches OpenAI for India to expand sovereign AI capabilities, accelerate enterprise adoption, and upskill the workforce, with India hosting OpenAI-led initiatives alongside Tata Group. ([openai.com](https://openai.com/openai-for-india))\n\n- Sovereign AI infrastructure: As part of Stargate, OpenAI and Tata will develop local AI-ready data-center capacity designed for data residency and security, with OpenAI becoming the first customer of Tata Consultancy Services’ HyperVault. 100 MW initial capacity, potential to scale to 1 GW. ([openai.com](https://openai.com/openai-for-india))\n\n- 100 MW buildout and scale plan: In the initial phase, Tata Group’s HyperVault unit will develop AI infrastructure with 100 MW capacity, with an option to scale to 1 GW to support next-generation AI workloads. ([tata.com](https://www.tata.com/newsroom/business/tata-openai-partnership))\n\n- Enterprise ChatGPT deployment: Tata Group plans to deploy ChatGPT Enterprise across its employees over the coming years, starting with hundreds of thousands of TCS employees. ([openai.com](https://openai.com/openai-for-india))\n\n- Codex integration for software: TCS will leverage OpenAI’s Codex to standardize AI-native software development across teams. ([openai.com](https://openai.com/openai-for-india))\n\n- Agentic AI focus: The partnership will develop industry-specific agentic AI solutions by combining OpenAI’s agentic AI capabilities with TCS domain knowledge. ([tata.com](https://www.tata.com/newsroom/business/tata-openai-partnership))\n\n- Joint GTM and deployment: TCS and OpenAI will jointly enable Indian and global enterprises to deploy, integrate, and scale OpenAI platforms, driving AI-led transformation at scale. ([tata.com](https://www.tata.com/newsroom/business/tata-openai-partnership))\n\n- Upskilling and certifications: OpenAI Certifications will expand in India with TCS as the first participating organization outside the United States; more than 100,000 ChatGPT Edu licenses will be provided to help workforce skills. ([openai.com](https://openai.com/openai-for-india))\n\n- Social impact commitments: OpenAI Foundation and TCS will collaborate to provide AI training/resources to Indian youth and develop NGO toolkits, aiming to improve livelihoods for at least one million Indian youth. ([tata.com](https://www.tata.com/newsroom/business/tata-openai-partnership))\n\n- OpenAI presence in India: OpenAI plans to open new offices in Mumbai and Bengaluru later in 2026 to support local users and partners. ([openai.com](https://openai.com/openai-for-india))'},
 {'id': 213,
  'event_id': '1276_2026-02-20',
  'news_date': '2026-02-20',
  'output': '- General Catalyst commits $5 billion to invest in India over the next five years, announced at the India AI Impact Summit 2026 in New Delhi on February 19, 2026. ([techcrunch.com](https://techcrunch.com/2026/02/19/general-catalyst-commits-5b-to-india-over-five-years/))\n- This $5 billion pledge represents a fivefold increase from GC’s prior India earmark of about $500 million to $1 billion. ([techcrunch.com](https://techcrunch.com/2026/02/19/general-catalyst-commits-5b-to-india-over-five-years/))\n- The investment will target sectors including artificial intelligence, healthcare, defense technology, fintech, and consumer technology. ([techcrunch.com](https://techcrunch.com/2026/02/19/general-catalyst-commits-5b-to-india-over-five-years/))\n- The announcement comes less than two years after General Catalyst merged with local firm Venture Highway. ([techcrunch.com](https://techcrunch.com/2026/02/19/general-catalyst-commits-5b-to-india-over-five-years/))\n- Neeraj Arora serves as GC’s CEO for India, the Middle East & North Africa to lead the expanded push. ([techcrunch.com](https://techcrunch.com/2026/02/19/general-catalyst-commits-5b-to-india-over-five-years/))\n- GC is pursuing a platform-led model that combines traditional investing with a “creation” strategy, including roll-ups of companies in India. ([m.economictimes.com](https://m.economictimes.com/tech/technology/general-catalysts-hemant-taneja-on-5-billion-india-bet-and-its-shift-beyond-venture-capital-to-company-creation-in-the-ai-age/articleshow/128589856.cms))\n- GC views India’s biggest AI opportunity as large-scale real-world deployment rather than frontier-model development. ([techcrunch.com](https://techcrunch.com/2026/02/19/general-catalyst-commits-5b-to-india-over-five-years/))\n- GC’s India portfolio already includes Zepto, PB Health, Raphe, Jeh Aerospace, Pronto and Ayr Energy, with additional investments in Spinny, Farmart, and Loop Health noted in coverage. ([techcrunch.com](https://techcrunch.com/2026/02/19/general-catalyst-commits-5b-to-india-over-five-years/))\n- The commitment coincides with broad activity at the India AI Impact Summit, which has attracted participation from global players and AI initiatives (e.g., OpenAI, Anthropic, Google among others). ([techcrunch.com](https://techcrunch.com/2026/02/19/general-catalyst-commits-5b-to-india-over-five-years/))'},
 {'id': 214,
  'event_id': '1277_2026-02-20',
  'news_date': '2026-02-20',
  'output': '- WHAT IS NEW: Abu Dhabi’s G42 and MBZUAI, partnered with Cerebras and India’s C-DAC, announced an 8-exaflop national AI supercomputer to be deployed in India, announced on the sidelines of the India AI Impact Summit 2026. ([mbzuai.ac.ae](https://mbzuai.ac.ae/news/uae-to-deploy-8-exaflop-supercomputer-in-india-to-strengthen-local-sovereign-ai-infrastructure/))\n\n- HOSTING AND GOVERNANCE: The system will be hosted in India and operate under India-defined governance with data remaining within national jurisdiction, as part of strengthening sovereign AI infrastructure under the India AI Mission. ([mbzuai.ac.ae](https://mbzuai.ac.ae/news/uae-to-deploy-8-exaflop-supercomputer-in-india-to-strengthen-local-sovereign-ai-infrastructure/))\n\n- COMPUTE CAPACITY: The announced system is designed to deliver 8 exaflops of compute capacity, marking a step to exaflop-scale AI infrastructure in India. ([mbzuai.ac.ae](https://mbzuai.ac.ae/news/uae-to-deploy-8-exaflop-supercomputer-in-india-to-strengthen-local-sovereign-ai-infrastructure/))\n\n- PARTNER ROLES: Delivery will be by G42 and Cerebras, with MBZUAI and India’s C-DAC as collaborating partners. ([mbzuai.ac.ae](https://mbzuai.ac.ae/news/uae-to-deploy-8-exaflop-supercomputer-in-india-to-strengthen-local-sovereign-ai-infrastructure/))\n\n- ANNOUNCEMENT CONTEXT: The project was unveiled at the India AI Impact Summit 2026 in New Delhi. ([thenationalnews.com](https://www.thenationalnews.com/future/technology/2026/02/20/abu-dhabis-g42-and-mbzuai-join-us-firm-cerebras-to-build-india-ai-supercomputer/))\n\n- ACCESS AND USERS: Once operational, the supercomputer will be accessible to universities, startups, SMEs, and government ministries, aiming for broad public/private sector AI enablement. ([mbzuai.ac.ae](https://mbzuai.ac.ae/news/uae-to-deploy-8-exaflop-supercomputer-in-india-to-strengthen-local-sovereign-ai-infrastructure/))\n\n- PURPOSE AND IMPACT: The initiative targets accelerating both training and inference for large-scale AI models tailored to Indian needs. ([techcrunch.com](https://techcrunch.com/2026/02/20/uaes-g42-teams-up-with-cerebras-to-deploy-8-exaflops-of-compute-in-india/))\n\n- RELEVANT PREVIOUS COLLABORATION: MBZUAI and G42 previously released the Hindi-English language model NANDA 87B (Dec 2025), highlighting prior collaboration between the parties. ([mbzuai.ac.ae](https://mbzuai.ac.ae/news/uae-to-deploy-8-exaflop-supercomputer-in-india-to-strengthen-local-sovereign-ai-infrastructure/))\n\n- GEOPOLITICAL CONTEXT: The project follows broader India–UAE strategic alignment, including the December 2025 India–UAE Strategic Dialogue and the January 2026 visit of UAE leadership to India. ([thenationalnews.com](https://www.thenationalnews.com/future/technology/2026/02/20/abu-dhabis-g42-and-mbzuai-join-us-firm-cerebras-to-build-india-ai-supercomputer/))\n\n'},
 {'id': 215,
  'event_id': '1278_2026-02-20',
  'news_date': '2026-02-20',
  'output': '- Core development: Sarvam AI unveiled two indigenous LLMs, Sarvam-30B and Sarvam-105B, announced at the India AI Impact Summit 2026 in New Delhi, aimed at real-time use and advanced reasoning; MoE architecture with large context windows; open-sourcing planned. ([techcrunch.com](https://techcrunch.com/2026/02/18/indian-ai-lab-sarvams-new-models-are-a-major-bet-on-the-viability-of-open-source-ai/))\n\n- Model architecture: Both models use a mixture-of-experts design that activates only a fraction of parameters at inference to improve efficiency. ([techcrunch.com](https://techcrunch.com/2026/02/18/indian-ai-lab-sarvams-new-models-are-a-major-bet-on-the-viability-of-open-source-ai/))\n\n- Real-time capabilities: 30B targets production-ready, real-time conversational use with a 32,000-token context window; 105B targets more complex, multi-step reasoning with a 128,000-token window. ([techcrunch.com](https://techcrunch.com/2026/02/18/indian-ai-lab-sarvams-new-models-are-a-major-bet-on-the-viability-of-open-source-ai/))\n\n- Training from scratch: 30B was pre-trained from scratch on about 16 trillion tokens; 105B trained on trillions of tokens across multiple Indian languages. ([techcrunch.com](https://techcrunch.com/2026/02/18/indian-ai-lab-sarvams-new-models-are-a-major-bet-on-the-viability-of-open-source-ai/))\n\n- IndiaAI Mission and compute: Training leveraged India’s government-backed IndiaAI Mission compute, with infrastructure support from Yotta and technical support from Nvidia. ([techcrunch.com](https://techcrunch.com/2026/02/18/indian-ai-lab-sarvams-new-models-are-a-major-bet-on-the-viability-of-open-source-ai/))\n\n- Open-source plan: Sarvam stated it intends to open-source the 30B and 105B models, though it did not specify whether training data or full training code would be disclosed. ([techcrunch.com](https://techcrunch.com/2026/02/18/indian-ai-lab-sarvams-new-models-are-a-major-bet-on-the-viability-of-open-source-ai/))\n\n- Enterprise and ecosystem: The launch outlined an expanded enterprise stack, including Sarvam for Work and a conversational agent platform called Samvaad, plus a vision model for document understanding. ([techcrunch.com](https://techcrunch.com/2026/02/18/indian-ai-lab-sarvams-new-models-are-a-major-bet-on-the-viability-of-open-source-ai/))\n\n- Indus consumer app: Two days after the model announcements, Sarvam released Indus, a consumer AI chat app powered by the 105B model, available in beta on iOS, Android, and the web; initial rollout limited to India with a waitlist. ([techcrunch.com](https://techcrunch.com/2026/02/20/indias-sarvam-launches-indus-ai-chat-app-as-competition-heats-up/?utm_source=openai))\n\n- Benchmark and language focus: The 105B model is positioned to perform on benchmarks relevant to Indian languages and enterprise tasks, with claims of competitive performance against open/closed models of similar size. ([business-standard.com](https://www.business-standard.com/technology/tech-news/sarvam-105b-model-sovereign-ai-india-foundation-model-launch-impact-summit-126021900551_1.html?utm_source=openai))\n\n- Industry coverage: Coverage of the announcement and related products appeared in TechCrunch (Feb 18, 2026), Economic Times (ETtech, Feb 18, 2026), Fortune India (Feb 18, 2026), and The Times of India (Feb 19, 2026), among others. ([techcrunch.com](https://techcrunch.com/2026/02/18/indian-ai-lab-sarvams-new-models-are-a-major-bet-on-the-viability-of-open-source-ai/))'},
 {'id': 216,
  'event_id': '1279_2026-02-20',
  'news_date': '2026-02-20',
  'output': '- Fractal launches Vaidya 2.0, the next generation of its healthcare reasoning models, available at Vaidya.ai. ([fractal.ai](https://fractal.ai/about-us/media/fractal-launches-vaidya-2.0))\n\n- Vaidya 2.0 scores 50.1 on OpenAI HealthBench (hard), and Fractal says it is the first AI model to exceed 50 on this benchmark, outperforming GPT-5 and Google’s Gemini Pro 3. ([fractal.ai](https://fractal.ai/about-us/media/fractal-launches-vaidya-2.0))\n\n- Debuted at the India AI Impact Summit 2026 in New Delhi, with HealthBench (hard) performance highlighted among its claims. ([fractal.ai](https://fractal.ai/about-us/media/fractal-launches-vaidya-2.0))\n\n- Fractal describes Vaidya 2.0 as designed to power a “Health Care Operating System” of workflows that bridge raw data to healthcare action. ([fractal.ai](https://fractal.ai/about-us/media/fractal-launches-vaidya-2.0))\n\n- Key citizen-facing use cases highlighted for Vaidya 2.0 include Emergency Assist (rapid triage and decision support), Symptom Checker, and Patient Journey Assist. ([fractal.ai](https://fractal.ai/about-us/media/fractal-launches-vaidya-2.0))\n\n- The model shows strong performance on the MedExpert benchmark and introduces capabilities to support Doctor Assist, in addition to OpenAI HealthBench (hard) Health Data Tasks for administrators. ([fractal.ai](https://fractal.ai/about-us/media/fractal-launches-vaidya-2.0))\n\n- Fractal is positioned as part of India’s sovereign AI push under the ₹10,300+ crore India AI Mission, framing Vaidya 2.0 as the first in a family of verticalized foundation models for the Global South. ([fractal.ai](https://fractal.ai/about-us/media/fractal-launches-vaidya-2.0))\n\n- Vaidya 2.0 is described as a post-trained, reasoning- and agentic-based healthcare model rather than a pure knowledge-based foundation model. ([fractal.ai](https://fractal.ai/about-us/media/fractal-launches-vaidya-2.0))\n\n- The release reiterates that Vaidya 2.0 is being demonstrated at the India AI Impact Summit 2026, with the broader Vaidya suite showcased at the event. ([fractal.ai](https://fractal.ai/about-us/media/fractal-launches-vaidya-2.0))'},
 {'id': 217,
  'event_id': '1280_2026-02-20',
  'news_date': '2026-02-20',
  'output': '- Core development: Stanford HAI and AWS launched the Stanford and AWS Marketing Science Lab to advance AI-driven marketing measurement, including measurement techniques, causal inference, and AI-based measurement tools. ([linkedin.com](https://www.linkedin.com/posts/activity-7429646063973593088-KKyW?utm_source=openai))\n- Collaboration scope: The lab unites Stanford Institute for Human-Centered AI (HAI), Stanford Data Science, the Stanford Graduate School of Business, and Amazon Web Services (AWS). ([linkedin.com](https://www.linkedin.com/posts/activity-7429646063973593088-KKyW?utm_source=openai))\n- Leadership: The lab is led by Guido Imbens (HAI-SDS Co-Director), Susan Athey (GSB), Wesley Hartmann (GSB), and Jiafeng (Kevin) Chen (Economics). ([linkedin.com](https://www.linkedin.com/posts/activity-7429646063973593088-KKyW?utm_source=openai))\n- Announcement date: Public coverage and posts indicate the launch was announced around February 20, 2026. ([edtechinnovationhub.com](https://www.edtechinnovationhub.com/news/stanford-hai-and-aws-launch-marketing-science-lab-focused-on-ai-measurement?utm_source=openai))\n- Focus areas: The initiative concentrates on marketing measurement, AI-powered measurement, causal inference methods, and analysis of B2B customer journeys. ([linkedin.com](https://www.linkedin.com/posts/activity-7429646063973593088-KKyW?utm_source=openai))\n- Expected outputs: Outcomes include published research, open-source code, and prototypes. ([linkedin.com](https://www.linkedin.com/posts/activity-7429646063973593088-KKyW?utm_source=openai))\n- Relevance to AI product measurement: The lab is described as an academic–industry initiative relevant to AI product measurement and adoption. ([edtechinnovationhub.com](https://www.edtechinnovationhub.com/news/stanford-hai-and-aws-launch-marketing-science-lab-focused-on-ai-measurement?utm_source=openai))\n- Cloud-scale research angle: The collaboration emphasizes scalable tools and cloud-enabled experimentation, leveraging AWS infrastructure. ([edtechinnovationhub.com](https://www.edtechinnovationhub.com/news/stanford-hai-and-aws-launch-marketing-science-lab-focused-on-ai-measurement?utm_source=openai))\n- Notable acknowledgement: Julia White, AWS Vice President and Chief Marketing Officer, is credited with helping spur the collaboration. ([linkedin.com](https://www.linkedin.com/posts/activity-7429646063973593088-KKyW?utm_source=openai))'},
 {'id': 218,
  'event_id': '1284_2026-02-20',
  'news_date': '2026-02-20',
  'output': '- Gemini 3.1 Pro is announced as a smarter Gemini model, with an upgraded core intelligence designed for tasks where a simple answer isn’t enough. It’s described as a step forward in core reasoning for complex problem-solving. ([blog.google](https://blog.google/innovation-and-ai/models-and-research/gemini-models/gemini-3-1-pro/))\n\n- Rollout is moving from today across multiple product strands: developers in preview via the Gemini API (including Google AI Studio), Gemini CLI, Google Antigravity, and Android Studio; enterprises via Vertex AI and Gemini Enterprise; consumers via the Gemini app and NotebookLM. ([blog.google](https://blog.google/innovation-and-ai/models-and-research/gemini-models/gemini-3-1-pro/))\n\n- Access to 3.1 Pro is available through the Gemini API, Vertex AI, the Gemini app, and NotebookLM. ([blog.google](https://blog.google/innovation-and-ai/models-and-research/gemini-models/gemini-3-1-pro/))\n\n- In benchmarking, 3.1 Pro achieved a verified ARC-AGI-2 score of 77.1%, described as more than double the reasoning performance of 3 Pro. ([blog.google](https://blog.google/innovation-and-ai/models-and-research/gemini-models/gemini-3-1-pro/))\n\n- Code-based animation capability: 3.1 Pro can generate website-ready, animated SVGs directly from a text prompt. ([blog.google](https://blog.google/innovation-and-ai/models-and-research/gemini-models/gemini-3-1-pro/))\n\n- Complex system synthesis example: the model can bridge complex APIs to user-friendly designs, demonstrated by building a live aerospace dashboard that visualizes the ISS orbit. ([blog.google](https://blog.google/innovation-and-ai/models-and-research/gemini-models/gemini-3-1-pro/))\n\n- Interactive design example: 3.1 Pro can code a complex 3D starling murmuration with hand-tracking and a generative score that shifts based on movement. ([blog.google](https://blog.google/innovation-and-ai/models-and-research/gemini-models/gemini-3-1-pro/))\n\n- Overall framing: 3.1 Pro is a smarter baseline for complex problem solving and is being released in preview to validate updates and advance agentic workflows across consumer, developer, and enterprise products. ([blog.google](https://blog.google/innovation-and-ai/models-and-research/gemini-models/gemini-3-1-pro/))'},
 {'id': 219,
  'event_id': '1286_2026-02-20',
  'news_date': '2026-02-20',
  'output': '- OpenAI and Microsoft joined the UK’s AI Security Institute’s international coalition to safeguard AI development, with OpenAI pledging funding to the Alignment Project; total funding now over £27 million to support about 60 projects. ([gov.uk](https://www.gov.uk/government/news/openai-and-microsoft-join-uks-international-coalition-to-safeguard-ai-development))\n\n- The Alignment Project is an international, UK-led effort to advance AI alignment research to ensure frontier AI systems are safe, secure, and under human control. ([gov.uk](https://www.gov.uk/government/news/openai-and-microsoft-join-uks-international-coalition-to-safeguard-ai-development))\n\n- OpenAI is contributing £5.6 million to the Alignment Project, alongside ongoing support from Microsoft and other partners, contributing to the £27 million+ total funding. ([gov.uk](https://www.gov.uk/government/news/openai-and-microsoft-join-uks-international-coalition-to-safeguard-ai-development))\n\n- The coalition supporting the Alignment Project includes a broad set of partners beyond OpenAI/Microsoft, such as CIFAR (Canadian Institute for Advanced Research), Australia’s AI Safety Institute, Schmidt Sciences, AWS, Anthropic, and several funds and UK bodies. ([gov.uk](https://www.gov.uk/government/news/openai-and-microsoft-join-uks-international-coalition-to-safeguard-ai-development))\n\n- The Alignment Project combines grant funding for research, compute infrastructure access, and ongoing academic mentorship from the AI Security Institute to accelerate alignment work. ([gov.uk](https://www.gov.uk/government/news/openai-and-microsoft-join-uks-international-coalition-to-safeguard-ai-development))\n\n- First Alignment Project grants have been awarded to 60 projects spanning eight countries, with a second funding round expected to open in the summer. ([gov.uk](https://www.gov.uk/government/news/openai-and-microsoft-join-uks-international-coalition-to-safeguard-ai-development))\n\n- Announcements were made by UK Deputy Prime Minister David Lammy and AI Minister Kanishka Narayan at the AI Impact Summit in India, marking the close of that summit. ([gov.uk](https://www.gov.uk/government/news/openai-and-microsoft-join-uks-international-coalition-to-safeguard-ai-development))\n\n- Mia Glaese, OpenAI VP of Research, emphasized that advancing alignment requires collaboration across independent teams and that OpenAI’s support complements its internal alignment work. ([gov.uk](https://www.gov.uk/government/news/openai-and-microsoft-join-uks-international-coalition-to-safeguard-ai-development))\n\n- The program is framed as a mechanism to build public trust in AI and enable safer deployment of frontier AI technologies in public services and national renewal. ([gov.uk](https://www.gov.uk/government/news/openai-and-microsoft-join-uks-international-coalition-to-safeguard-ai-development))'},
 {'id': 220,
  'event_id': '1287_2026-02-20',
  'news_date': '2026-02-20',
  'output': '- WHAT IS NEW — On February 20, 2026, PhonePe launched an AI-powered natural language search feature built on Microsoft Foundry that lets users initiate and complete in-app tasks, including payments, via voice or text. ([phonepe.com](https://www.phonepe.com/press/phonepe-launches-ai-powered-search-built-using-microsoft-foundry/))\n- INTENT-BASED ROUTING — The feature replaces traditional navigation with intent-based routing, directing users to their desired action (e.g., "Pay Hemanth 20 rupees" pre-selects the recipient or surfaces contacts named Hemanth). ([phonepe.com](https://www.phonepe.com/press/phonepe-launches-ai-powered-search-built-using-microsoft-foundry/))\n- ACTION SURFACING — Commands like “Recharge FASTag” or “Gold price” navigate to the most relevant page or surface the appropriate options. ([phonepe.com](https://www.phonepe.com/press/phonepe-launches-ai-powered-search-built-using-microsoft-foundry/))\n- PRIVACY/COMPUTE ARCHITECTURE — It uses a hybrid model of on-device and cloud inferencing, with personal and transactional data kept within the PhonePe environment. ([phonepe.com](https://www.phonepe.com/press/phonepe-launches-ai-powered-search-built-using-microsoft-foundry/))\n- ROLLOUT DETAIL — The rollout is being implemented in phases across India, with access via the Global Search Bar, Help Center, and History tab in the PhonePe app. ([phonepe.com](https://www.phonepe.com/press/phonepe-launches-ai-powered-search-built-using-microsoft-foundry/))\n- OFFICIAL VOICES — Rahul Chari (PhonePe CTO) described the launch as moving toward intent-based intelligent interfaces; Microsoft’s Puneet Chandok commented on Foundry’s role. ([phonepe.com](https://www.phonepe.com/press/phonepe-launches-ai-powered-search-built-using-microsoft-foundry/))\n- TECHNOLOGY PARTNER — The AI capability is powered by Microsoft Foundry technology. ([phonepe.com](https://www.phonepe.com/press/phonepe-launches-ai-powered-search-built-using-microsoft-foundry/))\n- CONTEXTUAL BACKDROP — The launch coincides with India’s AI ecosystem activity, with ET Online situating it around the AI Impact Summit 2026. ([economictimes.indiatimes.com](https://economictimes.indiatimes.com/industry/banking/finance/phonepe-rolls-out-ai-powered-feature-to-pay-via-voice-or-text/articleshow/128601184.cms?from=mdr))\n- COMPANY SCALE CONTEXT — PhonePe notes over 65 crore registered users and more than 4.7 crore merchants as of September 2025. ([phonepe.com](https://www.phonepe.com/press/phonepe-launches-ai-powered-search-built-using-microsoft-foundry/))'},
 {'id': 221,
  'event_id': '1288_2026-02-20',
  'news_date': '2026-02-20',
  'output': '- What is new: OpenAI has introduced Lockdown Mode for ChatGPT (an optional, advanced security setting) along with Elevated Risk labels for certain capabilities; the feature was announced on February 13, 2026, with consumer rollout planned in coming months. ([openai.com](https://openai.com/index/introducing-lockdown-mode-and-elevated-risk-labels-in-chatgpt/?utm_source=openai))\n\n- Who it’s for: Lockdown Mode is designed for a small set of high-security users (e.g., executives, security teams) and is not required by most users; it is available to ChatGPT Enterprise, Edu, Healthcare, and Teachers, with admins able to enable it in Workspace Settings. ([openai.com](https://openai.com/index/introducing-lockdown-mode-and-elevated-risk-labels-in-chatgpt/?utm_source=openai))\n\n- Core purpose: The mode aims to mitigate prompt-injection risks and data exfiltration by tightening how ChatGPT interacts with external systems. ([openai.com](https://openai.com/index/introducing-lockdown-mode-and-elevated-risk-labels-in-chatgpt/?utm_source=openai))\n\n- Web browsing in Lockdown Mode: Web browsing is limited to cached content, preventing live network requests from leaving OpenAI’s controlled network. This is intended to reduce potential data exfiltration via browsing. ([openai.com](https://openai.com/index/introducing-lockdown-mode-and-elevated-risk-labels-in-chatgpt/?utm_source=openai))\n\n- Other interaction constraints: Lockdown Mode deterministically disables certain tools and capabilities that could be exploited by adversaries; it tightens how ChatGPT can access apps and external sources. (OpenAI describes these as broad restrictions beyond standard protections.) ([openai.com](https://openai.com/index/introducing-lockdown-mode-and-elevated-risk-labels-in-chatgpt/?utm_source=openai))\n\n- Availability and administration: Admins can tailor which apps and actions are allowed under Lockdown Mode for individual users within a workspace; OpenAI notes a consumer rollout is planned for the coming months. ([openai.com](https://openai.com/index/introducing-lockdown-mode-and-elevated-risk-labels-in-chatgpt/?utm_source=openai))\n\n- Elevated Risk labels: A separate feature assigns “Elevated Risk” labels to specific capabilities across ChatGPT, ChatGPT Atlas, and Codex to communicate higher-risk use cases and guide user decisions. ([openai.com](https://openai.com/index/introducing-lockdown-mode-and-elevated-risk-labels-in-chatgpt/?utm_source=openai))\n\n- Rationale and protections: The rollout builds on existing safeguards (sandboxing, URL-data exfiltration protections, monitoring, audit logs, and enterprise controls) to reduce risk when features interact with networks and external apps. ([openai.com](https://openai.com/index/introducing-lockdown-mode-and-elevated-risk-labels-in-chatgpt/?utm_source=openai))\n\n- Forbes interpretation: Forbes’ analysis frames Lockdown Mode as being “repurposed” to limit problematic mental-health outputs, illustrating how the feature could affect safety in specific use cases. ([forbes.com](https://www.forbes.com/sites/lanceeliot/2026/02/20/new-chatgpt-lockdown-mode-repurposed-to-stop-ai-from-giving-out-bad-mental-health-advice/?utm_source=openai))\n\n- Additional coverage and context: Other outlets (e.g., Yahoo Tech and Indian Express) summarize Lockdown Mode as a security upgrade that pairs with Elevated Risk labels to improve transparency about feature risks. ([tech.yahoo.com](https://tech.yahoo.com/ai/chatgpt/articles/chatgpt-now-lockdown-mode-enable-160050855.html/?utm_source=openai))'},
 {'id': 222,
  'event_id': '1289_2026-02-20',
  'news_date': '2026-02-20',
  'output': '- Meta plans to launch its first smartwatch in 2026, code-named Malibu 2, featuring health tracking and a built-in Meta AI assistant. ([aol.com](https://www.aol.com/articles/meta-reboots-smartwatch-plan-aims-232800969.html?utm_source=openai))\n- The smartwatch revival follows a 2022 shutdown of a prior Meta smartwatch effort amid Reality Labs cost cuts; the project traces back roughly five years and included camera-equipped concepts. ([aol.com](https://www.aol.com/articles/meta-reboots-smartwatch-plan-aims-232800969.html?utm_source=openai))\n- The device is slated for release later in 2026, according to the initial reporting on the Malibu 2 project. ([aol.com](https://www.aol.com/articles/meta-reboots-smartwatch-plan-aims-232800969.html?utm_source=openai))\n- It would compete directly with established smartwatches such as the Apple Watch and other wearables. ([forbes.com](https://www.forbes.com/sites/andrewwilliams/2026/02/19/meta-smartwatch-makes-perfect-sense-amid-smart-glasses-strategy/?utm_source=openai))\n- Reported features include health-tracking capabilities and integration with Meta’s AI, rather than a generic, non-AI smartwatch. ([aol.com](https://www.aol.com/articles/meta-reboots-smartwatch-plan-aims-232800969.html?utm_source=openai))\n- The Malibu 2 concept is discussed in the context of broader Meta wearables strategy, potentially enabling cross-device interactions with Ray-Ban smart glasses. ([forbes.com](https://www.forbes.com/sites/andrewwilliams/2026/02/19/meta-smartwatch-makes-perfect-sense-amid-smart-glasses-strategy/?utm_source=openai))\n- Meta has not publicly commented on the Malibu 2 plan when approached by reporters. ([aol.com](https://www.aol.com/articles/meta-reboots-smartwatch-plan-aims-232800969.html?utm_source=openai))\n- The wearables backdrop includes Ray-Ban Display glasses, which reportedly shipped close to 6 million units in 2025, underscoring a growing AI-enabled wearable ecosystem. ([aol.com](https://www.aol.com/articles/meta-reboots-smartwatch-plan-aims-232800969.html?utm_source=openai))\n- Separately, Meta reportedly delayed its Phoenix mixed-reality glasses to 2027 and paused international rollout of Ray-Ban Display glasses to prioritize US demand, signaling a staggered hardware roadmap. ([businessinsider.com](https://www.businessinsider.com/meta-delays-new-mixed-reality-glasses-code-named-phoenix-2025-12?utm_source=openai))'},
 {'id': 223,
  'event_id': '1291_2026-02-20',
  'news_date': '2026-02-20',
  'output': '- Core development: Lyria 3, Google DeepMind’s latest generative music model, is rolling out in beta inside the Gemini app, enabling creation of 30-second tracks from text prompts, photos, or videos, with auto lyrics and optional cover art; output is watermark-tagged via SynthID. ([blog.google](https://blog.google/innovation-and-ai/products/gemini-app/lyria-3?utm_source=openai))\n\n- Global beta availability and rollout: The feature is available to Gemini users 18+, with desktop access today and mobile rollout in the coming days; supported languages include English, German, Spanish, French, Hindi, Japanese, Korean, and Portuguese. ([blog.google](https://blog.google/innovation-and-ai/products/gemini-app/lyria-3?utm_source=openai))\n\n- Output length and scope: Lyria 3 targets 30-second tracks, designed for short-form content (e.g., YouTube Shorts, TikTok, Reels). ([techcrunch.com](https://techcrunch.com/2026/02/18/google-adds-music-generation-capabilities-to-the-gemini-app/?utm_source=openai))\n\n- Auto lyrics capability: Lyrics generation is built into the flow, removing the previous requirement to supply lyrics manually. ([blog.google](https://blog.google/innovation-and-ai/products/gemini-app/lyria-3?utm_source=openai))\n\n- Multimodal control: Users can steer generation with text plus visual prompts (images/videos) to influence genre, mood, tempo, and vocal tone. ([blog.google](https://blog.google/innovation-and-ai/products/gemini-app/lyria-3?utm_source=openai))\n\n- Copyright and safety safeguards: Outputs are filtered to discourage direct artist mimicry, with provenance tools and a reporting channel for rights concerns; SynthID watermarking supports identification of AI-generated audio. ([blog.google](https://blog.google/innovation-and-ai/products/gemini-app/lyria-3?utm_source=openai))\n\n- YouTube Dream Track integration: Lyria 3 is being extended to YouTube creators via Dream Track, expanding access beyond Gemini to creator workflows. ([techcrunch.com](https://techcrunch.com/2026/02/18/google-adds-music-generation-capabilities-to-the-gemini-app/?utm_source=openai))\n\n- Creator-facing features (art and branding): Gemini generates custom cover art for tracks (e.g., via Nano Banana) to accompany AI-made songs. ([techcrunch.com](https://techcrunch.com/2026/02/18/google-adds-music-generation-capabilities-to-the-gemini-app/?utm_source=openai))\n\n- Availability and access tier details: Free usage exists with plan-based limits; Google AI Plus, Pro, and Ultra subscribers are described as receiving higher generation ceilings. ([blog.google](https://blog.google/innovation-and-ai/products/gemini-app/lyria-3?utm_source=openai))'},
 {'id': 224,
  'event_id': '1292_2026-02-20',
  'news_date': '2026-02-20',
  'output': '- Qumis announces an oversubscribed seed round of $4.3 million led by MTech Capital, with American Family Ventures joining as a new strategic investor; total funding now sits at $6.75 million. ([globenewswire.com](https://www.globenewswire.com/news-release/2026/02/19/3241222/0/en/Qumis-Raises-4-3M-to-Bring-Attorney-Grade-Coverage-Intelligence-to-Commercial-Insurance.html?utm_source=openai))\n\n- The round will be deployed to expand Qumis’ go-to-market team and deepen product capabilities around coverage intelligence for commercial insurance. ([globenewswire.com](https://www.globenewswire.com/news-release/2026/02/19/3241222/0/en/Qumis-Raises-4-3M-to-Bring-Attorney-Grade-Coverage-Intelligence-to-Commercial-Insurance.html?utm_source=openai))\n\n- Qumis’ platform is described as attorney-trained AI for commercial insurance coverage intelligence that reads and reasons across complex policy documents to deliver structured, citation-backed insights with transparent reasoning. ([globenewswire.com](https://www.globenewswire.com/news-release/2026/02/19/3241222/0/en/Qumis-Raises-4-3M-to-Bring-Attorney-Grade-Coverage-Intelligence-to-Commercial-Insurance.html?utm_source=openai))\n\n- Traction includes adoption by large brokers and carriers, notably NFP (an Aon company), with usage expanding from an initial team to hundreds of users across the organization. ([globenewswire.com](https://www.globenewswire.com/news-release/2026/02/19/3241222/0/en/Qumis-Raises-4-3M-to-Bring-Attorney-Grade-Coverage-Intelligence-to-Commercial-Insurance.html?utm_source=openai))\n\n- The funding comes amid continuing AI investment in insurance, with most capital historically flowing to workflow automation and document processing—Qumis is pursuing a more specialized, coverage-focused AI approach. ([globenewswire.com](https://www.globenewswire.com/news-release/2026/02/19/3241222/0/en/Qumis-Raises-4-3M-to-Bring-Attorney-Grade-Coverage-Intelligence-to-Commercial-Insurance.html?utm_source=openai))\n\n- Qumis is Chicago-based, founded in 2022, and built by coverage attorneys and AI experts to replace manual policy review with attorney-grade coverage insights. ([globenewswire.com](https://www.globenewswire.com/news-release/2026/02/19/3241222/0/en/Qumis-Raises-4-3M-to-Bring-Attorney-Grade-Coverage-Intelligence-to-Commercial-Insurance.html?utm_source=openai))\n\n- Lead investor in the round is MTech Capital; American Family Ventures is a new strategic investor; all prior investors participated in this round. ([globenewswire.com](https://www.globenewswire.com/news-release/2026/02/19/3241222/0/en/Qumis-Raises-4-3M-to-Bring-Attorney-Grade-Coverage-Intelligence-to-Commercial-Insurance.html?utm_source=openai))\n\n- The platform targets insurance teams across brokers, carriers, and coverage-focused law firms, offering policy analysis, policy comparisons, and real-time insights to support underwriting and claims. ([globenewswire.com](https://www.globenewswire.com/news-release/2026/02/19/3241222/0/en/Qumis-Raises-4-3M-to-Bring-Attorney-Grade-Coverage-Intelligence-to-Commercial-Insurance.html?utm_source=openai))\n\n- No valuation for the seed round was disclosed in the public announcements. ([globenewswire.com](https://www.globenewswire.com/news-release/2026/02/19/3241222/0/en/Qumis-Raises-4-3M-to-Bring-Attorney-Grade-Coverage-Intelligence-to-Commercial-Insurance.html?utm_source=openai))'},
 {'id': 225,
  'event_id': '1294_2026-02-20',
  'news_date': '2026-02-20',
  'output': '- OpenAI is developing a family of AI-powered devices, including a smart speaker and possibly smart glasses and a smart lamp, with a team of more than 200 people, signaling a shift from cloud/model-only products toward consumer hardware. ([channelnewsasia.com](https://www.channelnewsasia.com/business/openai-developing-ai-devices-including-smart-speaker-information-reports-5943236?utm_source=openai))\n- The smart speaker would be the first device in the lineup, with an estimated retail price of $200–$300 and not expected to ship before February 2027 at the earliest. ([channelnewsasia.com](https://www.channelnewsasia.com/business/openai-developing-ai-devices-including-smart-speaker-information-reports-5943236?utm_source=openai))\n- The speaker is described as including a camera to gather information about users and their surroundings. ([channelnewsasia.com](https://www.channelnewsasia.com/business/openai-developing-ai-devices-including-smart-speaker-information-reports-5943236?utm_source=openai))\n- The smart glasses in the plan are not expected to reach mass production until 2028. ([channelnewsasia.com](https://www.channelnewsasia.com/business/openai-developing-ai-devices-including-smart-speaker-information-reports-5943236?utm_source=openai))\n- The device family reportedly also includes a smart lamp as part of the initial concept set. ([channelnewsasia.com](https://www.channelnewsasia.com/business/openai-developing-ai-devices-including-smart-speaker-information-reports-5943236?utm_source=openai))\n- OpenAI’s hardware push traces back to its $6.5 billion acquisition of io Products, the Jony Ive–founded hardware unit. ([bloomberg.com](https://www.bloomberg.com/news/articles/2025-05-21/openai-to-buy-apple-veteran-jony-ive-s-ai-device-startup-in-6-5-billion-deal?utm_source=openai))\n- The move is framed as OpenAI aiming to capitalize on growing demand for physical AI and augmented reality rather than solely cloud/software offerings. ([channelnewsasia.com](https://www.channelnewsasia.com/business/openai-developing-ai-devices-including-smart-speaker-information-reports-5943236?utm_source=openai))\n- Industry context cited in reports notes Meta’s Ray-Ban smart glasses have had success, and that Apple and Google are also pursuing wearables. ([channelnewsasia.com](https://www.channelnewsasia.com/business/openai-developing-ai-devices-including-smart-speaker-information-reports-5943236?utm_source=openai))'}]

In [101]:
claude_article_builder(a)

{'headline': 'Nvidia Ditches $100B Chip Deal for $30B OpenAI Stake', 'output': "Nvidia is swapping its massive $100 billion multi-year chip supply agreement for a cleaner $30 billion direct equity investment in OpenAI. This shift makes Nvidia an actual owner, not just a supplier, in the AI giant currently valued at $830 billion. OpenAI's hunting for over $100 billion total this round, with SoftBank and Amazon joining the party. Plot twist: OpenAI plans to spend much of that fresh cash buying Nvidia chips anyway. This move signals Nvidia wants skin in the game beyond hardware sales, positioning itself as a stakeholder in AI's future rather than just the picks-and-shovels provider during the gold rush.", 'sources': [{'name': 'The Star', 'url': 'https://www.thestar.com.my/tech/tech-news/2026/02/20/nvidia-openai-near-30-billion-investment-in-place-of-unfinished-100-billion-deal-ft-reports'}], 'model_provider': 'claude', 'news_date': '2026-02-20', 'event_id': '1274_2026-02-20'}
✔️ Saved!
{'

In [100]:
def save_article(article_json):
    supabase.table('hundred_word_articles').insert({
        'event_id': article_json['event_id'],
        'model_provider': article_json['model_provider'],
        'news_date': article_json['news_date'],
        'sources': json.dumps(article_json['sources']),
        'output': article_json['output'],
        'headline': article_json['headline']
    }).execute()

In [80]:
final_summary_7

[{'headline': "Anthropic's AI Grading Bot Hits 80% Accuracy in Indian Schools",
  'output': "Anthropic and Pratham Education Foundation are deploying Claude-powered Anytime Testing Machines across Indian schools, turning handwritten answers into graded feedback in Hindi and English. The AI generates curriculum-aligned questions, scans student responses, grades them, and delivers personalized insights. Currently piloted in 20 schools with 1,500+ assessments, grading accuracy jumped from 30% to 80% against expert benchmarks. The program targets 100 schools by late 2026, including Pratham's Second Chance initiative supporting women preparing for India's Grade 10 exams. As India becomes Claude's second-largest market, AI-powered assessment could finally solve the classroom grading bottleneck at scale.",
  'sources': [{'name': 'India Today',
    'url': 'https://www.indiatoday.in/technology/news/story/anthropic-opens-india-office-announces-new-partnerships-at-india-ai-impact-summit-2869010-2

In [81]:
for i in final_summary_7:
    if i['news_date'] >= '2026-02-17':
        save_article(i)
        print("✔️ Saved!")

✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!


In [25]:
for i in final_summary_3:
    save_article(i)
    print("✔️ Saved!")

✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!


In [249]:

fail = []
pas = 0

for i in final_summary_3:
    words = i['output'].split()
    word_count = len(words)
    if word_count > 100:
        print(f"Fail: {word_count}")
        fail.append(word_count)
    else:
        print(f"Pass: {word_count}")
        pas +=1

print(f"Fail: {sum(fail)/len(fail)}")



Pass: 92
Pass: 93
Pass: 98
Pass: 95
Pass: 100
Pass: 99
Pass: 87
Pass: 91
Pass: 92
Pass: 97
Pass: 97
Pass: 100
Fail: 103
Pass: 92
Pass: 89
Pass: 92
Pass: 86
Fail: 101
Pass: 94
Pass: 89
Fail: 101
Fail: 110
Pass: 96
Fail: 105
Pass: 94
Fail: 102
Fail: 101
Pass: 83
Fail: 103.28571428571429


In [223]:
import statistics as stats